In [21]:
from pyspark.sql import pandas
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set("spark.sql.execution.arrow.enabled","true")

spark = SparkSession.builder \
.appName("EDU_REGIOES") \
.config(conf=conf) \
.getOrCreate()

sqlCtx = SQLContext(spark)

22/06/17 17:49:10 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
/usr/local/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning
22/06/17 17:49:10 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [22]:
from pyspark.sql.types import *
from pyspark.sql.functions import sum, max, min, avg, count, mean, round, col
from pyspark.sql.window import Window

In [23]:
%run -i funcoes.ipynb

s3 = Aws_importer()
f_pandas = Pandas_Spark()

In [24]:
#importing
import pandas as pd
pdf = pd.read_excel("s3a://alex-group2-sptech-bucket-brutos/ATU_BRASIL_REGIOES_UFS_2021.xlsx")

In [25]:
pdf

,Ano,Unidade Geográfica,Localização,Dependência Administrativa,Total 1,Creche,Pré-Escola,Total,Anos Iniciais,Anos Finais,...,7° ano,8° ano,9° ano,"Turmas Multietapa, Multi ou Correção de Fluxo 2",Total,1ª série,2ª série,3ª série,4ª série,Não-Seriado
0,2021,Brasil,Total,Total,15.4,13.4,17.2,22.9,21.2,26.6,...,26.9,26.9,26.6,15,30.1,30.8,30.4,29.7,21.9,23.9
1,2021,Brasil,Urbana,Total,15.4,13.3,17.6,24.2,21.9,27.7,...,27.9,28,27.9,16.8,30.7,31.2,30.9,30.3,21.7,26.3
2,2021,Brasil,Rural,Total,14.8,14.9,14.6,17.1,16.4,20.2,...,21.1,20.3,19.3,14.9,23,24.2,23.6,21.9,25.7,16.7
3,2021,Brasil,Total,Federal,12.7,11.5,13.8,24.7,22,26,...,25.8,26.3,27.5,--,29.8,31.6,30.7,29.2,25.8,24.7
4,2021,Brasil,Urbana,Federal,12.8,11.6,13.8,24.7,22.1,26,...,25.9,26.4,27.6,--,29.8,31.5,30.8,29.3,25.2,25.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,2021,Distrito Federal,Urbana,Privada,13.7,13.6,13.8,20.2,17.2,26.1,...,25.7,26.6,27.6,13,32.4,32.5,32.4,32.4,--,20.5
583,2021,Distrito Federal,Rural,Privada,19.4,22.6,12.3,28.6,8,38.9,...,36.3,36.4,44,--,40.4,48,26,50.5,--,--
584,2021,Distrito Federal,Total,Pública,19.6,13.3,19.7,24.1,21.2,29,...,29.8,29.9,29.9,9.6,33.8,32.9,34.3,34.7,--,11
585,2021,Distrito Federal,Urbana,Pública,19.8,13.8,19.9,24.4,21.4,29.2,...,30.1,30.3,30.2,6.6,34,33.1,34.5,35,--,11


In [26]:
pdf.columns

Index(['Ano', 'Unidade Geográfica', 'Localização',
       'Dependência Administrativa', 'Total 1', 'Creche', 'Pré-Escola',
       'Total', 'Anos Iniciais', 'Anos Finais', '1º Ano', '2° ano', '3° ano',
       '4° ano', '5° ano', '6° ano', '7° ano', '8° ano', '9° ano',
       'Turmas Multietapa, Multi ou Correção de Fluxo 2', 'Total ', '1ª série',
       '2ª série', '3ª série', '4ª série', 'Não-Seriado'],
      dtype='object')

In [27]:
pdf.dtypes

Ano                                                 int64
Unidade Geográfica                                 object
Localização                                        object
Dependência Administrativa                         object
Total 1                                            object
Creche                                             object
Pré-Escola                                         object
Total                                              object
Anos Iniciais                                      object
Anos Finais                                        object
1º Ano                                             object
2° ano                                             object
3° ano                                             object
4° ano                                             object
5° ano                                             object
6° ano                                             object
7° ano                                             object
8° ano        

In [28]:
pdf.count()

Ano                                                587
Unidade Geográfica                                 587
Localização                                        587
Dependência Administrativa                         587
Total 1                                            587
Creche                                             587
Pré-Escola                                         587
Total                                              587
Anos Iniciais                                      587
Anos Finais                                        587
1º Ano                                             587
2° ano                                             587
3° ano                                             587
4° ano                                             587
5° ano                                             587
6° ano                                             587
7° ano                                             587
8° ano                                             587
9° ano    

In [29]:
pdf.replace("--", 0, inplace=True)

In [31]:
pdf = pdf.astype({'1º Ano': float, '2° ano': float, '3° ano': float, '4° ano': float, '5° ano': float, '6° ano': float, '7° ano': float, '8° ano': float, '9° ano': float, '1ª série': float, '2ª série': float, '3ª série': float, '4ª série': float})

In [32]:
#dataframe from pandar to spark
sparkDF= f_pandas.pandas_to_spark(pdf)

/usr/local/lib/python3.7/site-packages/pyspark/sql/pandas/conversion.py:329: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [33]:
sparkDF.show(n=10)

+----+------------------+-----------+--------------------------+-------+------+----------+-----+-------------+-----------+------+------+------+------+------+------+------+------+------+-----------------------------------------------+------+--------+--------+--------+--------+-----------+
| Ano|Unidade Geográfica|Localização|Dependência Administrativa|Total 1|Creche|Pré-Escola|Total|Anos Iniciais|Anos Finais|1º Ano|2° ano|3° ano|4° ano|5° ano|6° ano|7° ano|8° ano|9° ano|Turmas Multietapa, Multi ou Correção de Fluxo 2|Total |1ª série|2ª série|3ª série|4ª série|Não-Seriado|
+----+------------------+-----------+--------------------------+-------+------+----------+-----+-------------+-----------+------+------+------+------+------+------+------+------+------+-----------------------------------------------+------+--------+--------+--------+--------+-----------+
|2021|            Brasil|      Total|                     Total|   15.4|  13.4|      17.2| 22.9|         21.2|       26.6|  19.8|  20

In [34]:
sparkDF.columns

['Ano',
 'Unidade Geográfica',
 'Localização',
 'Dependência Administrativa',
 'Total 1',
 'Creche',
 'Pré-Escola',
 'Total',
 'Anos Iniciais',
 'Anos Finais',
 '1º Ano',
 '2° ano',
 '3° ano',
 '4° ano',
 '5° ano',
 '6° ano',
 '7° ano',
 '8° ano',
 '9° ano',
 'Turmas Multietapa, Multi ou Correção de Fluxo 2',
 'Total ',
 '1ª série',
 '2ª série',
 '3ª série',
 '4ª série',
 'Não-Seriado']

In [36]:
sparkDF.select("1º Ano","2° ano","3° ano","4° ano","5° ano","6° ano","7° ano","8° ano","9° ano").summary("mean").show()

22/06/17 17:50:22 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|           1º Ano|           2° ano|            3° ano|            4° ano|            5° ano|            6° ano|           7° ano|            8° ano|            9° ano|
+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|   mean|15.74361158432709|16.64565587734242|17.154855195911406|18.256558773424196|18.550425894378204|22.056899488926756|22.92402044293015|22.764565587734243|22.519420783645657|
+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+



In [37]:
sparkDF = sparkDF.drop(*("Total 1","Total ", "Turmas Multietapa, Multi ou Correção de Fluxo 2","Não-Seriado","Creche","Pré-Escola","Total","Anos Iniciais","Anos Finais"))

In [38]:
#preprocessed
s3.save_s3_data(sparkDF,'cliente','MEDIAS_ESCOLARES_BRASIL')
s3.save_s3_data(sparkDF,'preprocessado','MEDIAS_ESCOLARES_BRASIL')
s3.save_data_client_side(sparkDF,'MEDIAS_ESCOLARES_BRASIL')

22/06/17 17:50:33 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 17:50:33 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 17:50:33 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 17:50:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 17:50:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 17:50:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [39]:
sparkDF.columns

['Ano',
 'Unidade Geográfica',
 'Localização',
 'Dependência Administrativa',
 '1º Ano',
 '2° ano',
 '3° ano',
 '4° ano',
 '5° ano',
 '6° ano',
 '7° ano',
 '8° ano',
 '9° ano',
 '1ª série',
 '2ª série',
 '3ª série',
 '4ª série']

In [40]:
sparkDF.printSchema

<bound method DataFrame.printSchema of DataFrame[Ano: bigint, Unidade Geográfica: string, Localização: string, Dependência Administrativa: string, 1º Ano: double, 2° ano: double, 3° ano: double, 4° ano: double, 5° ano: double, 6° ano: double, 7° ano: double, 8° ano: double, 9° ano: double, 1ª série: double, 2ª série: double, 3ª série: double, 4ª série: double]>

In [41]:
sugestionDF = sparkDF.groupBy("Ano","Unidade Geográfica","Dependência Administrativa","Localização").agg(round(avg("1º Ano"),1).alias("1 ano"),round(avg("2° Ano"),1).alias("2 ano"),\
                round(avg("3° Ano"),1).alias("3 ano"),round(avg("4° Ano"),1).alias("4 ano"),round(avg("5° Ano"),1).alias("5 ano"),round(avg("6° Ano"),1).alias("6 ano"),\
                round(avg("7° Ano"),1).alias("7 ano"),round(avg("8° Ano"),1).alias("8 ano"),round(avg("9° Ano"),1).alias("9 ano"),round(avg("1ª série"),1).alias("1 serie"),\
                round(avg("2ª série"),1).alias("2 serie"),round(avg("3ª série"),1).alias("3 serie"),round(avg("4ª série"),1).alias("4 serie")) \
        .where(sparkDF["Dependência Administrativa"]!="Total").where(sparkDF["Localização"]!="Total").orderBy("Unidade Geográfica")

In [42]:
sugestionDF.count()

324

In [44]:
sugestionDF.show(n=10)

+----+------------------+--------------------------+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+
| Ano|Unidade Geográfica|Dependência Administrativa|Localização|1 ano|2 ano|3 ano|4 ano|5 ano|6 ano|7 ano|8 ano|9 ano|1 serie|2 serie|3 serie|4 serie|
+----+------------------+--------------------------+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+
|2021|              Acre|                   Pública|      Rural| 18.2| 18.4| 19.1| 19.4| 19.2| 16.3| 18.0| 17.2| 17.5|   14.7|   17.5|   16.5|    0.0|
|2021|              Acre|                  Estadual|     Urbana| 23.2| 24.1| 25.5| 26.5| 26.4| 31.9| 31.7| 31.3| 31.9|   33.3|   33.8|   33.4|    0.0|
|2021|              Acre|                  Estadual|      Rural| 15.5| 16.1| 16.7| 17.8| 17.6| 16.3| 18.0| 17.0| 18.3|   14.7|   17.5|   16.5|    0.0|
|2021|              Acre|                 Municipal|      Rural| 20.0| 19.9| 20.6| 20.6| 20.5|

In [45]:
sugestionDF.printSchema

<bound method DataFrame.printSchema of DataFrame[Ano: bigint, Unidade Geográfica: string, Dependência Administrativa: string, Localização: string, 1 ano: double, 2 ano: double, 3 ano: double, 4 ano: double, 5 ano: double, 6 ano: double, 7 ano: double, 8 ano: double, 9 ano: double, 1 serie: double, 2 serie: double, 3 serie: double, 4 serie: double]>

In [46]:
_sugestionDF = sugestionDF.select(
    col('Unidade Geográfica').cast(StringType()).alias('UNIDADE_GEOGRAFICA'),
    col('Ano').cast(LongType()).alias('ANO'),
    col('Localização').cast(StringType()).alias('LOCALIZACAO'),
    col('Dependência Administrativa').cast(StringType()).alias('DEPENDENCIA_ADMINISTRATIVA'),
    col('1 ano').cast(FloatType()).alias('1_ANO'),
    col('2 ano').cast(FloatType()).alias('2_ANO'),
    col('3 ano').cast(FloatType()).alias('3_ANO'),
    col('4 ano').cast(FloatType()).alias('4_ANO'),
    col('5 ano').cast(FloatType()).alias('5_ANO'),
    col('6 ano').cast(FloatType()).alias('6_ANO'),
    col('7 ano').cast(FloatType()).alias('7_ANO'),
    col('8 ano').cast(FloatType()).alias('8_ANO'),
    col('9 ano').cast(FloatType()).alias('9_ANO'),
    col('1 serie').cast(FloatType()).alias('1_SERIE'),
    col('2 serie').cast(FloatType()).alias('2_SERIE'),
    col('3 serie').cast(FloatType()).alias('3_SERIE'),
    col('4 serie').cast(FloatType()).alias('4_SERIE'),
)

22/06/17 17:52:34 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/06/17 17:52:34 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/06/17 17:52:34 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/06/17 17:52:34 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/06/17 17:52:34 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the f

In [49]:
#saving into s3 client
s3.save_s3_data(sugestionDF,'preprocessado','MEDIAS_ESCOLARES_BRASIL/SUGESTAO')
s3.save_s3_data(sugestionDF,'cliente','MEDIAS_ESCOLARES_BRASIL/SUGESTAO')
s3.save_data_client_side(sugestionDF,'MEDIAS_ESCOLARES_BRASIL_sugestao')

22/06/17 18:04:20 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 18:04:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 18:04:23 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 18:04:24 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [39]:
#end the process of spark session
spark.stop()